## 7.2 Using stacks

Let's look at two classic applications of stacks.

### 7.2.1 Balanced brackets

Python expressions use round and square brackets. The brackets must be
balanced: opening and closing brackets must match in pairs, and
the brackets enclosed within a pair of matching brackets must also be balanced.
If the brackets are not balanced, a syntax error occurs.

In [1]:
('hello'[1:3 + 'h')] * 3    # brackets within () aren't balanced

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (2538324135.py, line 1)

Given a string, containing Python code or some other text,
we want to know if the brackets within that string are balanced.
That's a decision problem.

**Function**: is balanced?\
**Inputs**: *text*, a string\
**Preconditions**: each bracket in *text* is one of (, ), [, ]\
**Output**: *balanced*, a Boolean\
**Postconditions**: *balanced* is true if and only if

- all opening and closing brackets in *text* match (have the same shape) in pairs
- for each pair of matching brackets, the substring they delimit is balanced

For the test table we need balanced and unbalanced brackets, nested and not.
We must think of the various ways in which brackets are unbalanced.
The empty string, the absence of brackets, and a string only with brackets
are edge cases.

In [2]:
from algoesup import check_tests

is_balanced_tests = [
    # case,         text,                               balanced
    ['no text',     '',                                 True],
    ['no brackets', 'brackets are like Russian dolls',  True],
    ['matched',     '(3 + 4)',                          True],
    ['mismatched',  '(3 + 4]',                          False],
    ['not opened',  '3 + 4]',                           False],
    ['not closed',  '(3 + 4',                           False],
    ['wrong order', 'close ) before open (',            False],
    ['nested',      '([([])])',                         True],
    ['nested pair', 'items[(i - 1):(i + 1)]',           True]
]

check_tests(is_balanced_tests, [str, bool])

OK: the test table passed the automatic checks.


I told you at the start of this section that this problem requires a stack.
If I hadn't let the cat out of the bag, one way to spot a stack
is to look for a LIFO relationship. In a balanced string, the *first* closing
bracket must match the *last* (most recent) opening bracket, or
vice versa the *first* opening bracket must match the *last* closing bracket.

By the way, this problem is a particular case of the more general
problem of matching delimiters that can be nested.
Any language that has them, like the opening and closing tags in HTML and XML,
requires a stack to be processed.

<div class="alert alert-warning">
<strong>Note:</strong> When processing a nested structure, consider a stack.
</div>

The algorithm must check if each closing bracket matches the most recent
opening bracket. For that, we must keep a stack of the brackets opened so far.
As we go through the string, we push opening brackets on the stack,
and we pop them as we go through the matching closing brackets.
Let's work through the last test in the table, as it has both kinds of brackets
and they're nested. Characters that aren't brackets are skipped.

Stack | Remaining string | Action
-|-|-
&nbsp; | '[(i - 1):(i + 1)]'  |  opening bracket: push
[  | '(i - 1):(i + 1)]'  |  opening bracket: push
[(  | '):(i + 1)]'  |  closing bracket matches top item: pop
[  | '(i + 1)]'  |  opening bracket: push
[(  | ')]'  |  closing bracket matches top item: pop
[  | ']'  |  closing bracket matches top item: pop

We reached the end of the string. Since the stack is empty,
i.e. there are no open but yet unclosed brackets,
the brackets must be balanced.

The other tests with balanced brackets are checked in a similar way,
so let's consider the tests with unbalanced brackets.
As the algorithm goes through the string, how can it spot that the fourth text,
'(3 + 4]', isn't balanced?

___

When it reaches the closing square bracket, the stack has the opening round
bracket. The brackets don't match, so the algorithm can immediately stop
with *balanced* = false.

Let's consider now the 'not opened' case: `'3 + 4]'`.
How can the algorithm spot the brackets are unbalanced?

___

When it reaches the closing square bracket, the stack is empty.
There's no opening bracket the closing bracket can match, so the algorithm
can immediately stop with a false output.

The 'wrong order' case, `'close ) before open ('`, is processed
in the same way, so let's look at the final 'not closed' case: `'(3 + 4'`.
How can the algorithm detect it's unbalanced?

___

Upon reaching the end of the string, the stack isn't empty.
Some brackets were opened but not closed, which means they're unbalanced.

Now that we have thought through the test cases, we can outline the algorithm.
Here's one possible way, using the imperative tense and
describing some of the rationale.

> Create an empty stack for the brackets opened so far and not yet matched.
> Iterate over the string and process each character.
> If it's an opening bracket, push it on the stack.
> If it's a closing bracket and the stack is empty or the top item isn't a
> matching opening bracket, stop: the brackets aren't balanced.
> Otherwise, the brackets match, so pop the opening bracket from the stack.
> If the end of the string is reached, the brackets are balanced if and only if
> the stack is empty, i.e. no open brackets remain to be closed.

Here's the step-by-step algorithm.

1. let *opened* be an empty stack
2. for each *character* in *text*:
    1. if *character* = '(' or *character* = '[':
        1. push *character* on *opened*
    2. otherwise if *character* = ')':
        1. if │*opened*│ > 0 and top of *opened* = '(':
            1. pop *opened*
        2. otherwise:
            1. let *balanced* be false
            2. stop
    3. otherwise if *character* = ']':
        1. if │*opened*│ > 0 and top of *opened* = '[':
            1. pop *opened*
        2. otherwise:
            1. let *balanced* be false
            2. stop
3. let *balanced* be │*opened*│ = 0

#### Exercise 7.2.1

Give a best- and a worst-case scenario of the algorithm and
their corresponding complexities.

_Write your answer here._

[Hint](../31_Hints/Hints_07_2_01.ipynb)
[Answer](../32_Answers/Answers_07_2_01.ipynb)

#### Exercise 7.2.2

Translate the algorithm to Python code.

In [3]:
%run -i ../m269_stack

from algoesup import test

def is_balanced(text: str) -> bool:
    """Check if all brackets in text are balanced.

    Preconditions: each bracket in 'text' is one of (, ), [, ]
    """
    pass

test(is_balanced, is_balanced_tests)

[Answer](../32_Answers/Answers_07_2_02.ipynb)

#### Exercise 7.2.3

Assuming only round brackets (parentheses) can occur in the input string,
outline a simpler linear-time algorithm that checks if they're balanced.

[Hint](../31_Hints/Hints_07_2_03.ipynb)
[Answer](../32_Answers/Answers_07_2_03.ipynb)

### 7.2.2 Postfix expressions

An expression like 3 − 4 is in **infix** notation: the operator is
between the operands. An expression like 3 4 − is in **postfix** notation:
the operator comes after the operands.

Postfix expressions don't require parentheses because they show explicitly
in which order the operators are applied. Some examples:

Infix | Postfix |
-|-
3 − 4 × 5 |  3 4 5 × −
(3 − 4) × 5 |  3 4 − 5 ×
(3 − 4) × (5 − 6) | 3 4 − 5 6 − ×
3 − 4 × (5 − 6)  | 3 4 5 6 − × −

<div class="alert alert-info">
<strong>Info:</strong> Postfix notation is also known as reverse Polish notation.
</div>

In the following, postfix expressions only contain
natural numbers and the subtraction and multiplication operators.

#### Exercise 7.2.4

Postfix expressions can be evaluated in a single pass from left to right,
using a stack. Explain why.

_Write your answer here._

[Hint](../31_Hints/Hints_07_2_04.ipynb)
[Answer](../32_Answers/Answers_07_2_04.ipynb)

#### Exercise 7.2.5

Outline an algorithm that evaluates a postfix expression, given as
a non-empty sequence of characters '−' and '*' and natural numbers.
You can assume the sequence represents a valid postfix expression,
e.g. it doesn't start with an operator.

_Write your answer here._

[Hint](../31_Hints/Hints_07_2_05.ipynb)
[Answer](../32_Answers/Answers_07_2_05.ipynb)

#### Exercise 7.2.6

Implement the algorithm. You don't need to add tests.

In [4]:
%run -i ../m269_stack

from algoesup import test

def evaluate(expression: list) -> int:
    """Return the value of the given postfix expression.

    Preconditions:
    - each item in the input list is a natural number, '-' or '*'
    - the input represents a valid non-empty postfix expression
    """
    pass

evaluate_tests = [
    # case,               expression,                   value
    ["3 * 4",             [3, 4, "*"],                  12],
    ["3 - 4",             [3, 4, "-"],                  -1],
    ["3 - 4 * 5",         [3, 4, 5, "*", "-"],          -17],
    ["(3 - 4) * 5",       [3, 4, "-", 5, "*"],          -5],
    ["(3 - 4) * (5 - 6)", [3, 4, "-", 5, 6, "-", "*"],  1],
    ["no operation",      [4],                          4]
]

test(evaluate, evaluate_tests)

[Answer](../32_Answers/Answers_07_2_06.ipynb)

⟵ [Previous section](07_1_stack.ipynb) | [Up](07-introduction.ipynb) | [Next section](07_3_queue.ipynb) ⟶